In [252]:
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations
import pandas as pd
from lxml import html
from lxml import etree
from bs4 import BeautifulSoup
import json
import requests
import grequests

In [130]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [282]:
def get_data(html):
    soup = BeautifulSoup(html)
    div = soup.find('div', {'class': 'center-colm'})
    uik = div.find('h2').text
    uik_n = uik.split(' ')[-1]
    tik = dicti[uik]
    p = div.find_all('p')

    # Адрес избирательной комиссии
    span = div.find('span', {'id': 'address_ik'})
    ik_adress_pre = span.find('span').text.split('муниципальное образование')
    ik_adress = 'муниципальное образование' + ik_adress_pre[1]
    ik_place = span.text.split(', ')[-1]

    # Координаты избирательной комисии
    span = div.find('span', {'id': 'view_in_map_ik'})
    ik_lat = span['coordlat']
    ik_lon = span['coordlon']
    
    # Телефон избирательной комиссии
    ik_tel = p[2].text.split(': ')[1].replace('(', '').replace(')', '')

    # Срок окончания полномочий
    date_end = p[5].text.split(': ')[1]

    # Адрес помещения для голосования
    span = div.find('span', {'id': 'address_voteroom'})
    voteroom_address_pre = span.find('span').text.split('муниципальное образование')
    voteroom_address = 'муниципальное образование' + voteroom_address_pre[1]
    voteroom_place = span.text.split(', ')[-1]

    # Координаты помещения для голосования
    span = div.find('span', {'id': 'view_in_map_voteroom'})
    voteroom_lat = span['coordlat']
    voteroom_lon = span['coordlon']

    # Телефон помещения для голосования
    voteroom_tel = p[2].text.split(': ')[1].replace('(', '').replace(')', '')
    
    # Председатель
    try:
        chairperson = div.find_all('td')[1].text
    except:
        chairperson = 'ошибка'
    return [uik_n, tik, ik_adress, ik_place, ik_lat, ik_lon, ik_tel, date_end, voteroom_address, voteroom_place, voteroom_lat, voteroom_lon, voteroom_tel, chairperson, ik_adress == voteroom_address]

In [243]:
# Скачивание всех страниц
url = "http://www.st-petersburg.vybory.izbirkom.ru/region/st-petersburg?action=ik"
rs = (grequests.post(url, data = {'numik': str(uik_nm)}) for uik_nm in range(2500))
result = grequests.map(rs)

In [257]:
# УИК по id ТИК
def get_uiks(tik_id, tik_name):
    url = "http://www.st-petersburg.vybory.izbirkom.ru/region/st-petersburg?action=ikTree&region=78&vrn=" + tik_id + "&onlyChildren=true&id=" + tik_id
    data_uik = requests.request("GET", url)
    data_uik = json.loads(data_uik.text)

    for child in data_uik:
        dicti[child['text']] = tik_name

In [261]:
# Все ТИК
url = "http://www.st-petersburg.vybory.izbirkom.ru/region/st-petersburg?action=ikTree&region=78"
dicti = {}

response = requests.request("GET", url)
data = json.loads(response.text)
children = data[0].get('children')

for child in children:
    get_uiks(child['id'], child['text'])

In [283]:
# Генерация df из данных
columns = ['УИК', 'ТИК', 'Адрес ИК','Место ИК','ik_lat','ik_lon','Телефон ИК','Срок окончания полномочий','Адрес помещения голосования','Место помещения голосования','voteroom_lat','voteroom_lon','Телефон помещения голосования', 'Председатель', 'Совпадают адреса?']
df = pd.DataFrame (columns = columns)
for i in trange(len(result)):
    response = result[i]
    if text_error_request not in response.text: 
        res = get_data(response.text)
        df.loc[i] = res

df

,УИК,ТИК,Адрес ИК,Место ИК,ik_lat,ik_lon,Телефон ИК,Срок окончания полномочий,Адрес помещения голосования,Место помещения голосования,voteroom_lat,voteroom_lon,Телефон помещения голосования,Председатель,Совпадают адреса?
1,№1,ТИК № 1,муниципальное образование муниципальный округ ...,ГБОУ СОШ № 232,59.922152,30.297057,89313267472,08.06.2023,муниципальное образование муниципальный округ ...,ГБОУ СОШ № 232,59.922152,30.297057,89313267472,Алымов Михаил Александрович,True
2,№2,ТИК № 1,муниципальное образование муниципальный округ ...,ГБОУ СОШ №260,59.919364,30.295036,88127141668,08.06.2023,муниципальное образование муниципальный округ ...,ГБОУ СОШ №260,59.919364,30.295036,88127141668,Йылдыз Оксана Николаевна,True
3,№3,ТИК № 1,муниципальное образование муниципальный округ ...,ГБОУ СОШ №234,59.918940,30.286269,88124175311,08.06.2023,муниципальное образование муниципальный округ ...,ГБОУ СОШ №234,59.918940,30.286269,88124175311,Суслов Александр Владимирович,True
4,№4,ТИК № 1,муниципальное образование муниципальный округ ...,ГБОУ СОШ №234,59.917466,30.287688,88127144921,07.06.2023,муниципальное образование муниципальный округ ...,ГБОУ СОШ №234,59.917466,30.287688,88127144921,Кулына Елизавета Викторовна,True
5,№5,ТИК № 1,муниципальное образование муниципальный округ ...,ГБОУ СОШ № 260,59.919364,30.295036,88127141668,08.06.2023,муниципальное образование муниципальный округ ...,ГБОУ СОШ № 260,59.919364,30.295036,88127141668,Минеев Дмитрий Николаевич,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2367,№2367,ТИК № 24,муниципальное образование муниципальный округ ...,дом 56 корпус 1 (ГБОУ Школа №328),59.869815,30.451020,-,06.11.2025,муниципальное образование муниципальный округ ...,дом 56 корпус 1 (ГБОУ Школа №328),59.869815,30.451020,-,Омшев Александр Юрьевич,True
2395,№2395,ТИК № 56,муниципальное образование муниципальный округ ...,школа № 573,60.03801,30.21568,\n,02.11.2025,муниципальное образование муниципальный округ ...,школа № 573,60.03801,30.21568,\n,Иванова Татьяна Анатольевна,True
2396,№2396,ТИК № 56,муниципальное образование муниципальный округ ...,школа № 246,60.028757,30.229711,\n,02.11.2025,муниципальное образование муниципальный округ ...,школа № 246,60.028757,30.229711,\n,Шабанов Александр Викторович,True
2397,№2397,ТИК № 56,муниципальное образование муниципальный округ ...,школа № 632,60.057317,30.272351,\n,02.11.2025,муниципальное образование муниципальный округ ...,школа № 632,60.057317,30.272351,\n,Пурчел Оксана Александровна,True


C:\ProgramData\Anaconda3\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [284]:
df.to_excel('последний.xlsx', engine='xlsxwriter', index=False)